In [ ]:
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np

In [ ]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
pickle_files = ["./plot_data/hh.pickle", "./plot_data/memory_bound.pickle", "./plot_data/compute_bound.pickle"]

In [ ]:
results = {}
def load_pickle_result_file(pickle_files, results):
    for pickle_file in pickle_files:
        with open(pickle_file, 'rb') as handle:
            results.update(pickle.load(handle))

load_pickle_result_file(pickle_files, results)

In [ ]:
def _get_flags_string(flags):
    return flags.replace(" ", "_").replace('-','').replace('=','_')

In [ ]:
# colors = ['#f0f9e8','#bae4bc','#7bccc4','#43a2ca','#0868ac'] # print friendly colors
# sns.set_palette(sns.color_palette(colors))
# sns.set_palette("Set3") # NEURON paper palette
colors = ['#0570b0', '#66c2a4','#238b45','#fdd49e','#fc8d59','#9ebcda','#8c96c6','#969696','#525252']
# pick colors according to the following order which matches the order of 
# intel svml #0570b0
# gcc #66c2a4
# gcc_svml #238b45
# clang #fdd49e
# clang_svml #fc8d59
# mod2ir #9ebcda
# mod2ir_svml #8c96c6
# mor2it_jit_svml #969696
# mod2ir_jit_sleef #525252
sns.set_palette(sns.color_palette(colors))
def generate_graph_pandas(results, compilers_comparison_config, graph_suffix, output_dir, print_values=False):
    os.makedirs(output_dir, exist_ok=True)
    compiler_flags = json.loads(compilers_comparison_config)
    for modname in results:
        # state
        bar_data_state_cpu_panda = {}
        bar_data_state_cpu_panda["architecture"] = []
        bar_data_state_cpu_panda["compiler"] = []
        bar_data_state_cpu_panda["runtime"] = []
        # current
        bar_data_cur_cpu_panda = {}
        bar_data_cur_cpu_panda["architecture"] = []
        bar_data_cur_cpu_panda["compiler"] = []
        bar_data_cur_cpu_panda["runtime"] = []
        for architecture in results[modname]:
            for compiler in results[modname][architecture]:
                if compiler in compiler_flags and architecture in compiler_flags[compiler]:
                    for flags in compiler_flags[compiler][architecture]:
                        if compiler == "nmodl_jit":
                            state_kernel_name = "nrn_state_{}".format(modname.replace("-", "_"))
                            cur_kernel_name = "nrn_cur_{}".format(modname.replace("-", "_"))
                        else:
                            state_kernel_name = "nrn_state_ext"
                            cur_kernel_name = "nrn_cur_ext"
                        if compiler == "clang" and "jit" in flags:
                            compiler_name = "mod2ir"
                        elif compiler == "nmodl_jit":
                            compiler_name = "mod2ir_jit"
                        else:
                            compiler_name = compiler
                        if "svml" in flags or "SVML" in flags:
                            compiler_name = compiler_name + "_svml"
                        elif "sleef" in flags or "SLEEF" in flags:
                            compiler_name = compiler_name + "_sleef"
                        if architecture == "default":
                            architecture_label = "auto-scalar"
                        elif architecture == "nehalem":
                            architecture_label = "nehalem-sse2"
                        elif architecture == "broadwell":
                            architecture_label = "broadwell-avx2"
                        else: # skylake-avx512
                            architecture_label = architecture
                        bar_data_state_cpu_panda["architecture"].append(architecture_label)
                        bar_data_state_cpu_panda["compiler"].append(compiler_name)
                        if _get_flags_string(flags) not in results[modname][architecture][compiler]:
                            bar_data_state_cpu_panda["runtime"].append(0)
                        else:
                            bar_data_state_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0])
                        bar_data_cur_cpu_panda["architecture"].append(architecture_label)
                        bar_data_cur_cpu_panda["compiler"].append(compiler_name)
                        if _get_flags_string(flags) not in results[modname][architecture][compiler]:
                            bar_data_cur_cpu_panda["runtime"].append(0)
                        else:
                            bar_data_cur_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][cur_kernel_name][0])
        fig, axes = plt.subplots(1, 2, squeeze=False, figsize=(14,7))
        df_state = pd.DataFrame(bar_data_state_cpu_panda, columns=["architecture", "compiler", "runtime"])
        sns.barplot(x='architecture', y='runtime', hue='compiler', data=df_state, ax=axes[0,0])
        axes[0,0].xaxis.label.set_visible(False)
        axes[0,0].yaxis.label.set_visible(False)
        axes[0,0].set_title("nrn_state runtime for {}".format(modname))
        axes[0,0].get_legend().remove()
        if print_values:
          for i in axes[0,0].containers:
            axes[0,0].bar_label(i,)
        df_cur = pd.DataFrame(bar_data_cur_cpu_panda, columns=["architecture", "compiler", "runtime"])
        sns.barplot(x='architecture', y='runtime', hue='compiler', data=df_cur, ax=axes[0,1])
        axes[0,1].xaxis.label.set_visible(False)
        axes[0,1].yaxis.label.set_visible(False)
        axes[0,1].set_title("nrn_cur runtime for {}".format(modname))
        if print_values:
          for i in axes[0,1].containers:
            axes[0,1].bar_label(i,)
        fig.text(0.5, 0.04, 'Target Microarchitecture-Instruction Set', ha='center', va='center')
        fig.text(0.06, 0.5, 'Runtime (s)', ha='center', va='center', rotation='vertical')
        plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        plt.savefig("{}/{}_benchmark_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()

compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "clang": {
    "default": [
      "-O3 -ffast-math",
      "-O3 -ffast-math -fveclib=SVML",
      "-O3 -ffast-math jit SVML",
      "-O3 -ffast-math jit SLEEF"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp jit SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp jit SLEEF"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SLEEF"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SLEEF"
    ]
  },
  "gcc": {
    "default": [
      "-O3 -ffast-math -ftree-vectorize",
      "-O3 -ffast-math -ftree-vectorize -mveclibabi=svml"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ]
  }
}
"""

generate_graph_pandas(results, compilers_comparison_config, "cpu_all_compilers", "graphs_output_pandas")

In [ ]:
compilers_comparison_config = """
{
  "intel": {
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "nmodl_jit": {
    "nehalem": [
      "SVML_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn"
    ]
  }
}
"""
colors = ['#0570b0','#969696']
sns.set_palette(sns.color_palette(colors))

generate_graph_pandas(results, compilers_comparison_config, "cpu_intel_vs_nmodl", "graphs_output_pandas")

In [ ]:
def generate_graph_pandas_gpu(results, compilers_comparison_config, graph_suffix, output_dir, print_values=False):
    compiler_flags = json.loads(compilers_comparison_config)
    for modname in results:
        df = None
        bar_data_gpu_panda = {}
        bar_data_gpu_panda["kernel"] = []
        bar_data_gpu_panda["compiler"] = []
        bar_data_gpu_panda["runtime"] = []
        architecture = "nvptx64"
        for compiler in results[modname][architecture]:
            if compiler in compiler_flags and architecture in compiler_flags[compiler]:
                for flags in compiler_flags[compiler][architecture]:
                    dict_label = "{}_{}_{}".format(architecture, compiler, _get_flags_string(flags))
                    if compiler == "nmodl_jit":
                        state_kernel_name = "nrn_state_{}".format(modname.replace("-", "_"))
                        cur_kernel_name = "nrn_cur_{}".format(modname.replace("-", "_"))
                    else:
                        state_kernel_name = "nrn_state_ext"
                        cur_kernel_name = "nrn_cur_ext"
                    if compiler == "clang" and "jit" in flags:
                        compiler_name = "mod2ir"
                    elif compiler == "nmodl_jit":
                        compiler_name = "mod2ir_jit"
                    else:
                        compiler_name = compiler
                    bar_data_gpu_panda["kernel"].append("nrn_state")
                    bar_data_gpu_panda["compiler"].append(compiler_name)
                    if _get_flags_string(flags) not in results[modname][architecture][compiler]:
                        bar_data_gpu_panda["runtime"].append(0)
                    else:
                        bar_data_gpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0])
                    bar_data_gpu_panda["kernel"].append("nrn_current")
                    bar_data_gpu_panda["compiler"].append(compiler_name)
                    if _get_flags_string(flags) not in results[modname][architecture][compiler]:
                        bar_data_gpu_panda["runtime"].append(0)
                    else:
                        bar_data_gpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][cur_kernel_name][0])
        df_state = pd.DataFrame(bar_data_gpu_panda, columns=["kernel", "compiler", "runtime"])
        fig, ax = plt.subplots(figsize=(6,6))
        ax = sns.barplot(x='kernel', y='runtime', hue='compiler', data=df_state, ax=ax)
        if print_values:
            for i in ax.containers:
                ax.bar_label(i,)
        plt.xlabel("Kernel Name")
        plt.ylabel("Runtime (s)")
        plt.title("OpenACC and MOD2IR comparison for {}".format(modname))
        plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        plt.savefig("{}/{}_benchmark_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()

compilers_comparison_config = """
{
  "nvhpc": {
    "nvptx64": [
      "-O3 -gpu=nordc,fastmath"
    ]
  },
  "nmodl_jit": {
    "nvptx64": [
      "libdevice_nnancontractafn"
    ]
  }
}
"""
colors = ['#b2df8a','#bdbdbd']
sns.set_palette(sns.color_palette(colors))

generate_graph_pandas_gpu(results, compilers_comparison_config, "gpu_openacc_vs_nmodl", "graphs_output_pandas")

In [ ]:
colors = ['#0570b0','#66c2a4','#238b45','#fdd49e','#fc8d59','#9ebcda','#8c96c6','#969696','#525252']
# pick colors according to the following order which matches the order of 
# intel #023858
# gcc #66c2a4
# gcc_svml #238b45
# clang #fdd49e
# clang_svml #fc8d59
# mod2ir #9ebcda
# mod2ir_svml #8c96c6
# mor2it_jit_svml #969696
# mod2ir_jit_sleef #525252
sns.set_palette(sns.color_palette(colors))
compilers_comparison_config = """
{
  "intel": {
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "clang": {
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SLEEF"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SLEEF"
    ]
  },
  "nmodl_jit": {
    "broadwell": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ]
  }
}
"""

generate_graph_pandas(results, compilers_comparison_config, "cpu_intel_clang_mod2ir_compilers", "graphs_output_pandas")

In [ ]:
results_expsyn = {}
load_pickle_result_file(["./plot_data/expsyn_icc_clang.pickle"], results_expsyn)

In [ ]:
compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "clang": {
    "default": [
      "-O3 -ffast-math",
      "-O3 -ffast-math -fveclib=SVML",
      "-O3 -ffast-math jit SVML",
      "-O3 -ffast-math jit SLEEF"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp jit SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp jit SLEEF"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SLEEF"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SLEEF"
    ]
  },
  "gcc": {
    "default": [
      "-O3 -ffast-math -ftree-vectorize",
      "-O3 -ffast-math -ftree-vectorize -mveclibabi=svml"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ]
  }
}
"""
generate_graph_pandas(results_expsyn, compilers_comparison_config, "cpu_intel_clang_mod2ir_compilers_expsyn", "graphs_output_pandas")

In [ ]:
results_hh = {}
load_pickle_result_file(["./plot_data/hh_ic_clang_gcc_w_wout_svml.pickle"], results_hh)
colors = ['#6baed6', '#0570b0', '#66c2a4','#238b45','#fdd49e','#fc8d59','#9ebcda','#8c96c6','#969696','#525252']
# pick colors according to the following order which matches the order of 
# intel #6baed6
# intel svml #0570b0
# gcc #66c2a4
# gcc_svml #238b45
# clang #fdd49e
# clang_svml #fc8d59
# mod2ir #9ebcda
# mod2ir_svml #8c96c6
# mor2it_jit_svml #969696
# mod2ir_jit_sleef #525252
sns.set_palette(sns.color_palette(colors))
compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div",
      "-O2 -prec-div -fimf-use-svml"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div",
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div",
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fopenmp",
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml -fopenmp"
    ]
  },
  "gcc": {
    "default": [
      "-O3 -ffast-math -ftree-vectorize",
      "-O3 -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ]
  },
  "clang": {
    "default": [
      "-O3 -ffast-math",
      "-O3 -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -ffast-math jit SVML",
      "-O3 -ffast-math jit SLEEF"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp jit SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp jit SLEEF"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp jit SLEEF"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SLEEF"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ]
  }
}
"""
generate_graph_pandas(results_hh, compilers_comparison_config, "hh_icc_clang_gcc_w_wout_svml", "graphs_output_pandas")

In [ ]:
colors = ['#6baed6', '#0570b0', '#66c2a4','#238b45','#fdd49e','#fc8d59','#9ebcda','#8c96c6','#969696','#525252']
# pick colors according to the following order which matches the order of 
# intel #6baed6
# intel svml #0570b0
# gcc #66c2a4
# gcc_svml #238b45
# clang #fdd49e
# clang_svml #fc8d59
# mod2ir #9ebcda
# mod2ir_svml #8c96c6
# mor2it_jit_svml #969696
# mod2ir_jit_sleef #525252
sns.set_palette(sns.color_palette(colors))
compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div",
      "-O2 -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fopenmp",
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml -fopenmp"
    ]
  },
  "gcc": {
    "default": [
      "-O3 -ffast-math -ftree-vectorize",
      "-O3 -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ]
  },
  "clang": {
    "default": [
      "-O3 -ffast-math",
      "-O3 -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -ffast-math jit SVML",
      "-O3 -ffast-math jit SLEEF"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp jit SLEEF"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn",
      "SLEEF_nnancontractafn"
    ]
  }
}
"""
generate_graph_pandas(results_hh, compilers_comparison_config, "hh_icc_clang_gcc_w_wout_svml", "graphs_output_pandas")